In [97]:
import os
import yaml
import random
import numpy as np
import cv2

from pathlib import Path
from rosbags.highlevel import AnyReader

In [98]:
def print_bag_stats(bag):
    with AnyReader([Path(bag)]) as reader:
        for conn in reader.connections:
            print(f"Topic: {conn.topic}")
            print(f"  Type: {conn.msgtype}")

def compressed_image_to_rgb_array(msg):
    """
    Convert a sensor_msgs/CompressedImage to a HxWx3 uint8 RGB numpy array.
    """
    # msg.format can be 'jpeg', 'png', etc., but cv2.imdecode handles both.
    np_buf = np.frombuffer(msg.data, dtype=np.uint8)
    bgr = cv2.imdecode(np_buf, cv2.IMREAD_COLOR)
    if bgr is None:
        raise ValueError("Failed to decode CompressedImage (cv2.imdecode returned None).")
    rgb = cv2.cvtColor(bgr, cv2.COLOR_BGR2RGB)
    return rgb

def odom_msg_to_row(ts_ns, msg):
    """
    Convert a nav_msgs/Odometry message to a row:
    [ts_sec, x, y, z, qw, qx, qy, qz]
    """
    ts = ts_ns * 1e-9
    p = msg.pose.pose.position
    q = msg.pose.pose.orientation
    return np.array([ts, p.x, p.y, p.z, q.w, q.x, q.y, q.z], dtype=np.float64)


def process_bag(bag, bag_config):
    """
    bag - path for rosbag 
    bag_config - config dictionary listing which topics to extract.
      Expected shape:
      {
        "topics": [
          {"topic": "/camera/image/compressed"},
          {"topic": "/odom"},
          ...
        ]
      }
    Returns:
      {
        "images": { "<topic>": [ (ts_sec, rgb_np), ... ], ... },
        "odometry": { "<topic>": np.ndarray (N x 8) },
      }
    """
    topics_dict = bag_config.get("topics", [])
    topics = {topic_dict['topic']: topic_dict for topic_dict in topics_dict}
    names = []
    print_bag_stats(bag)

    # messages = 0
    outputs = { topic_dict['name'] : {} for topic_dict in topics_dict }
    with AnyReader([Path(bag)]) as reader:
        # Restrict to requested topics
        conns = {c.topic: c for c in reader.connections if c.topic in topics}
        if not conns:
            print("No matching topics found in config.")
            return outputs

        conn_to_topic = {c: t for t, c in conns.items()}
        print("Found topics:")
        for topic in conns:
            print(f"  {topic}")
        for conn, ts_ns, raw in reader.messages(connections=list(conns.values())):
            topic = conn_to_topic[conn]
            name = topics[topic]["name"]
            msg   = reader.deserialize(raw, conn.msgtype)

            ts_sec = ts_ns * 1e-9
            if hasattr(msg, 'header') and hasattr(msg.header, 'stamp'):
                stamp = msg.header.stamp
                ts_sec = stamp.sec + stamp.nanosec * 1e-9

            # Route based on type string (works for ROS1/ROS2 names)
            mtype = conn.msgtype
            if mtype.endswith('CompressedImage'):
                rgb = compressed_image_to_rgb_array(msg)
                outputs[name].setdefault('ts', []).append(ts_sec)
                outputs[name].setdefault('rgb', []).append(rgb)

            elif mtype.endswith('Odometry'):
                odom_np = odom_msg_to_row(ts_ns, msg)
                outputs[name].setdefault('odom', []).append(odom_np)
            
            # if messages > 1000:
            #     break
            # messages += 1

    # Stack lists into arrays where appropriate
    for name, data in outputs.items():
        for key, value in data.items():
            if isinstance(value, list):
                # For odometry and timestamps, stack into numpy arrays
                outputs[name][key] = np.stack(value) if value and isinstance(value[0], np.ndarray) else np.array(value)

    return outputs

In [99]:
DATASET="scand"
# Load yaml file
with open(f"../../configs/{DATASET}.yaml", "r") as f:
    bag_config = yaml.safe_load(f)

# Randomly select a bag from the directory
bag_dir = bag_config["data_dir"]
bag_paths = [Path(bag_dir) / f for f in os.listdir(bag_dir) if f.endswith('.bag')]
bag_paths = sorted(bag_paths, key=lambda p: int(p.stem))

# for bag_path in bag_paths:
#     print(f"Processing bag {bag_path}")
bag_outputs = process_bag(bag_paths[120], bag_config)

Topic: /localization
  Type: amrl_msgs/msg/Localization2DMsg
Topic: /navigation/cmd_vel
  Type: geometry_msgs/msg/Twist
Topic: /joystick
  Type: sensor_msgs/msg/Joy
Topic: /tf
  Type: tf2_msgs/msg/TFMessage
Topic: /joint_states
  Type: sensor_msgs/msg/JointState
Topic: /spot/camera/frontleft/camera_info
  Type: sensor_msgs/msg/CameraInfo
Topic: /spot/camera/frontleft/image/compressed
  Type: sensor_msgs/msg/CompressedImage
Topic: /spot/camera/frontright/image/compressed
  Type: sensor_msgs/msg/CompressedImage
Topic: /spot/camera/frontright/camera_info
  Type: sensor_msgs/msg/CameraInfo
Topic: /odom
  Type: nav_msgs/msg/Odometry
Topic: /scan
  Type: sensor_msgs/msg/LaserScan
Topic: /velodyne_points
  Type: sensor_msgs/msg/PointCloud2
Topic: /spot/camera/left/camera_info
  Type: sensor_msgs/msg/CameraInfo
Topic: /spot/camera/right/camera_info
  Type: sensor_msgs/msg/CameraInfo
Topic: /spot/camera/right/image/compressed
  Type: sensor_msgs/msg/CompressedImage
Topic: /spot/camera/left/imag

In [101]:
bag_outputs['front_camera'].keys()

dict_keys(['ts', 'rgb'])